#SQL TOKYO OLYMPIC PROJECT
**Satrting with data imortation from azure data lake 2 using spark.**
**Continuing with data transformation using pandas.**
**Then SQL queries on data.**


Usually I would connect directly through app registration in Azure (much faster) , however , since I have a student azure account I don't have the premmisions to do that.
So, I connected directly to the data lake through access key.

First I need to connect to the data lake azure storage:

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType , DoubleType , BooleanType, DataType 

In [ ]:
spark.conf.set(
"fs.azure.account.key.olympicdataofri.dfs.core.windows.net",
"SECRET KEY"
)

In [ ]:
dbutils.fs.ls("abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data")


[FileInfo(path='abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/EntriesGender.csv', name='EntriesGender.csv', size=1123, modificationTime=1713808216000),
 FileInfo(path='abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/Medals.csv', name='Medals.csv', size=2414, modificationTime=1713808216000),
 FileInfo(path='abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/athletes.csv', name='athletes.csv', size=418492, modificationTime=1713808216000),
 FileInfo(path='abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/coaches.csv', name='coaches.csv', size=16889, modificationTime=1713808215000),
 FileInfo(path='abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/teams.csv', name='teams.csv', size=35270, modificationTime=1713808214000)]

Loaded the files into data frame

In [ ]:
#set the data lake file location:
file_location1 = "abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/Medals.csv"
file_location2 = "abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/EntriesGender.csv"
file_location3 = "abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/athletes.csv"
file_location4 = "abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/coaches.csv"
file_location5 = "abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/teams.csv"

#read in the data to dataframe df
Medals = spark.read.format("csv").option("inferSchema", "true").option("header",
"true").option("delimiter",",").load(file_location1)
EntriesGender = spark.read.format("csv").option("inferSchema", "true").option("header",
"true").option("delimiter",",").load(file_location2)
athletes = spark.read.format("csv").option("inferSchema", "true").option("header",
"true").option("delimiter",",").load(file_location3)
coaches = spark.read.format("csv").option("inferSchema", "true").option("header",
"true").option("delimiter",",").load(file_location4)
teams = spark.read.format("csv").option("inferSchema", "true").option("header",
"true").option("delimiter",",").load(file_location5)

#display the dataframe
display(teams)


I want to check if the data is actually clean (even though it was taken *already* clean from kaggle)

In [ ]:
Medals.head()
Medals.printSchema()


root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [ ]:
EntriesGender.head()
EntriesGender.printSchema()


root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
athletes.head()
athletes.printSchema()


root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
coaches.head()
coaches.printSchema()


root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
teams.head()
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



Now start to work with sql , I need to create main data base and transform from dataframe to db:

In [ ]:
%sql
CREATE DATABASE Tokyoolympics

Created a local data brick table for each of the files. (in spark the data is immutable so i would have to filter the first row every query becouse it is a header)

In [ ]:
%sql 
CREATE TABLE Tokyoolympics.Teams (TeamName STRING,Discipline STRING,Country STRING,Event STRING)
USING CSV
LOCATION 'abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/teams.csv'

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS Tokyoolympics.EntriesGender (Discipline STRING, Female INT, Male INT,Total INT)
USING CSV
LOCATION 'abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/EntriesGender.csv'

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS Tokyoolympics.Medals (Rank INT, Team_Country STRING,Gold INT, Silver INT, Bronze INT, Total INT, Rank_Total INT)
USING CSV
LOCATION 'abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/Medals.csv'

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS Tokyoolympics.Athletes (PersonName STRING,Country STRING ,Discipline STRING )
USING CSV
LOCATION 'abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/athletes.csv'

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS Tokyoolympics.Coaches (Name STRING, Country STRING,Discipline STRING,Event STRING )
USING CSV
LOCATION 'abfss://olympic-data@olympicdataofri.dfs.core.windows.net/Clean-data/coaches.csv'

Check to see it loaded perfectly into the db

In [ ]:
%sql
select * from Tokyoolympics.teams where Event != 'Event'

TeamName,Discipline,Country,Event
Belgium,3x3 Basketball,Belgium,Men
China,3x3 Basketball,People's Republic of China,Men
China,3x3 Basketball,People's Republic of China,Women
France,3x3 Basketball,France,Women
Italy,3x3 Basketball,Italy,Women
Japan,3x3 Basketball,Japan,Men
Japan,3x3 Basketball,Japan,Women
Latvia,3x3 Basketball,Latvia,Men
Mongolia,3x3 Basketball,Mongolia,Women
Netherlands,3x3 Basketball,Netherlands,Men


# SQL queries 
**Showcase different insights using sql**


Find how many olympic athletes from each country and order them from max to min.

In [ ]:
%sql
select Country , count(*) as count from Tokyoolympics.athletes group by Country order by count  desc

Country,count
United States of America,615
Japan,586
Australia,470
People's Republic of China,401
Germany,400
France,377
Canada,368
Great Britain,366
Italy,356
Spain,324


Find which countries never won gold and then which countries never *not* won gold

In [ ]:
%sql
select * from Tokyoolympics.medals where Gold = 0

Rank,Team_Country,Gold,Silver,Bronze,Total,Rank_Total
66,Colombia,0,4,1,5,42
67,Azerbaijan,0,3,4,7,33
68,Dominican Republic,0,3,2,5,42
69,Armenia,0,2,2,4,47
70,Kyrgyzstan,0,2,1,3,60
71,Mongolia,0,1,3,4,47
72,Argentina,0,1,2,3,60
72,San Marino,0,1,2,3,60
74,Jordan,0,1,1,2,66
74,Malaysia,0,1,1,2,66


In [ ]:
%sql
select * from Tokyoolympics.medals where Gold > 0 and Silver = 0 and Bronze = 0 

Rank,Team_Country,Gold,Silver,Bronze,Total,Rank_Total
42,Bahamas,2,0,0,2,66
42,Kosovo,2,0,0,2,66
63,Bermuda,1,0,0,1,77
63,Morocco,1,0,0,1,77
63,Puerto Rico,1,0,0,1,77


Calculate the percentage of male and female athletes in each discipline from the EntriesGender table.

In [ ]:
%sql
select Discipline, (Female/Total*100) as Precentage from tokyoolympics.entriesgender where Discipline != 'Discipline' order by Precentage 

Discipline,Precentage
Wrestling,33.21799307958477
Cycling Road,34.82587064676617
Boxing,35.294117647058826
Equestrian,36.868686868686865
Baseball/Softball,38.46153846153847
Football,43.42105263157895
Water Polo,45.52238805970149
Swimming,46.34146341463415
Athletics,47.47672709456149
Cycling Track,47.61904761904761


Find all athletes and their respective coach

In [ ]:
%sql
SELECT athletes.PersonName as Athlete_Name, coaches.Name as Coach_name, athletes.Discipline as discipline
FROM tokyoolympics.athletes 
JOIN tokyoolympics.coaches
ON  athletes.Discipline = coaches.Discipline WHERE athletes.PersonName != 'PersonName'

Athlete_Name,Coach_name,discipline
ABALDE Alberto,ZAMORA PEDREIRA Javier,Basketball
ABALDE Tamara,ZAMORA PEDREIRA Javier,Basketball
ABALO Luc,VIVER ARZA Carlos Enrique,Handball
ABBINGH Lois,VIVER ARZA Carlos Enrique,Handball
ABBOTT Monica,YAMAJI Noriko,Baseball/Softball
ABDALLA Maryam,ZHANG Xiaohuan,Artistic Swimming
ABDALLAH Shahd,ZHANG Xiaohuan,Artistic Swimming
ABDELSALAM Mohamed,WIRTH Oscar,Football
ABDULHAMID Saud,WIRTH Oscar,Football
ABDULREDHA Mohamed,VIVER ARZA Carlos Enrique,Handball


Show coaches who are coaching more than one event (women/men/duet team) within the same discipline.

In [ ]:
%sql

SELECT A1.name , A1.Discipline , COUNT(DISTINCT A1.Event) AS NumberOfEvents
FROM tokyoolympics.coaches AS A1 JOIN tokyoolympics.coaches AS A2
ON A1.Name = A2.Name AND A1.Discipline = A2.Discipline
GROUP BY A1.Name , A1.Discipline
HAVING NumberOfEvents > 1


name,Discipline,NumberOfEvents
GIALLOMBARDO Patrizia,Artistic Swimming,2
SZAUDER Gabor,Artistic Swimming,2
FUJIKI Mayuko,Artistic Swimming,2
WANG Jie,Artistic Swimming,2
TARRES CAMPA Ana,Artistic Swimming,2
REQUENA PERICAS Judit,Artistic Swimming,2
FARINELLI Roberta,Artistic Swimming,2
CHERNETSKA Natalia,Artistic Swimming,2
RETEGUI Carlos Jose,Hockey,2
ELAFANDI Nour,Artistic Swimming,2


Determine the most efficient team in the world by calculating the ration of athlete to medals for each team:

How many athletes are in each team

In [ ]:
%sql
SELECT teams.TeamName, athletes.Country as cnt, COUNT(DISTINCT(athletes.PersonName)) as NumberOfAthletes
FROM tokyoolympics.teams
JOIN tokyoolympics.athletes ON teams.Country = athletes.Country
GROUP BY cnt , teams.TeamName

TeamName,cnt,NumberOfAthletes
SZUDI Adam / PERGEL Szandra,Hungary,154
Kantor/Losiak,Poland,195
Cuba,Cuba,69
Claes/Sponcil,United States of America,614
Azerbaijan,Azerbaijan,41
Krasilnikov/Stoyanovskiy,ROC,318
Germany,Germany,399
Huberli/Betschart,Switzerland,115
Abicha/Elgraoui,Morocco,48
Mexico,Mexico,155


Calculate the ratio of medals to athletes in each country 

In [ ]:
%sql

SELECT teams.TeamName, athletes.Country as cnt, 
COUNT(DISTINCT(athletes.PersonName)) as NumberOfAthletes , medals.total , 
CASE WHEN medals.Total >0 
THEN ROUND(COUNT(DISTINCT(athletes.PersonName)) / nullif(medals.total,0),2)
ELSE 0
END AS AthleteToMedalRatio
FROM tokyoolympics.teams
JOIN tokyoolympics.athletes ON teams.Country = athletes.Country 
JOIN tokyoolympics.medals ON teams.Country = medals.Team_Country
GROUP BY cnt , teams.TeamName , Total 
ORDER BY AthleteToMedalRatio DESC;

TeamName,cnt,NumberOfAthletes,total,AthleteToMedalRatio
Gallay/Pereyra,Argentina,180,3,60.0
Argentina,Argentina,180,3,60.0
Azaad/Capogrosso,Argentina,180,3,60.0
South Africa,South Africa,171,3,57.0
Abicha/Elgraoui,Morocco,48,1,48.0
Mexico,Mexico,155,4,38.75
Gaxiola/Rubio,Mexico,155,4,38.75
Lithuania,Lithuania,37,1,37.0
Puerto Rico,Puerto Rico,35,1,35.0
Saudi Arabia,Saudi Arabia,32,1,32.0


Link to my github (CV there) : https://github.com/ofriraichel/Data_Business_Analytics_Protfolio